In [4]:
import pandas as pd
import re
import numpy as np

df = pd.read_csv('properties_data.csv')
df = df.drop_duplicates(subset='id')

filtered_columns = ['id','location', 'Number of frontages', 'Type', 'Subtype', 
           'Price', 'Transaction Type', 'Bedrooms', 'Living area', 
           'Kitchen type', 'Furnished', 'How many fireplaces?', 'Terrace surface', 
           'Garden surface', 'Surface of the plot', 'Swimming pool', 'Building condition']

df = df[filtered_columns]

df = df.drop(df[df["Type"]=="house group"].index)
df = df.drop(df[df["Type"]=="apartment group"].index)

df = df.rename(columns={
    'location' :'Locality',
    'Transaction Type' : 'Type of sale',
    'Type' :'Type of property',
    'Subtype' : 'Subtype of property',
    'Number of frontages': 'Number of facades',
    'Bedrooms':'Number of rooms',
    'Surface of the plot' :'Surface of the land',
    'Garden surface':'Garden', 
    'Terrace surface':'Terrace',
    'Kitchen type' : 'Fully equipped kitchen',
    'How many fireplaces?' : 'Open fire'
})


def clean_and_convert(column):
    column = column.apply(lambda x: re.sub('\D+', '', str(x)))
    column = column.replace('', np.nan).fillna(0).astype(int)
    return column

df['Living area'] = clean_and_convert(df['Living area'])
df['Terrace'] = clean_and_convert(df['Terrace'])
df['Garden'] = clean_and_convert(df['Garden'])
df['Surface of the land'] = clean_and_convert(df['Surface of the land'])

#put the columns in a specific order
df = df.reindex(columns=['id', 'Locality', 'Type of property', 'Subtype of property',
                         'Type of sale', 'Price', 'Number of facades', 'Number of rooms', 
                         'Living area', 'Fully equipped kitchen', 'Furnished', 
                         'Surface of the land', 'Terrace', 'Garden', 'Open fire', 
                         'Swimming pool'])

df.to_csv('properties_clean.csv', index=False)

/tmp/ipykernel_1726/43969156.py:5: DtypeWarning: Columns (41,42,43,44,90,91,92,93,113,115) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('properties_data.csv')
